<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/10pandascombinacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introducción a la ciencia de datos con Python
### Rafa Caballero

### Combinando dataframes
Veamos en este Notebook cómo se combinan y agregan dataframes

### Índice
[Concatenar](#Concatenar)<br>
[Merge](#Merge)<br>



El siguiente código es para mostrar una dataframe al lado de otro, ejecutarlo pero no hace falta entenderlo

In [ ]:
from IPython.display import display, HTML
from IPython.display import display_html
from IPython.display import Markdown

def display_side_by_side(*args,title=""):
    print(title)
    html_str = ''
    for df in args:
        html_str += '&nbsp;&nbsp;&nbsp;'+df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'),
        raw=True)



<a name="Concatenar"></a>
### Concatenar

La forma más fácil, y a veces la más rápida y útil, de combinar dataframes, ya sea "pegándolo" debajo o al lado con `pd.concat`

<img src = "https://miro.medium.com/max/1050/1*0wu6DunCzPC4o9FIyRTW4w.png">

In [ ]:
import IPython.display as display
import pandas as pd
from pandas import DataFrame
from random import sample


df1 = DataFrame({'clave': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data': range(7),
                 'otro': sample(range(10, 30), 7)})

df2 = DataFrame({'clave': ['e', 'e', 'e', 'e'],
                 'data': range(4),
                 'otro': sample(range(10, 30), 4)})

display_side_by_side(df1,df2)

In [ ]:
df3 = pd.concat([df1,df2])
df3

Si nos incomoda que el índice no sea consecutivo:

In [ ]:
df3 = df3.reset_index(drop=True)
df3

Sin embargo, hay veces que es útil usar el índice (si no tiene ya otro cometido) para "apuntar" el origen de cada fila

In [ ]:
import IPython.display as display
import pandas as pd

df1 = DataFrame({'clave': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data': range(7),
                 'otro': sample(range(10, 30), 7)})

df2 = DataFrame({'clave': ['e', 'e', 'e', 'e'],
                 'data': range(4),
                 'otro': sample(range(10, 30), 4)})

df1.index = ["A"]*len(df1)
df2.index = ["B"]*len(df2)
display_side_by_side(df1,df2,pd.concat([df1,df2]),title="pd.concat([df1,df2])")

**Ojo** porque `concat` no es tan *tonto* como parece; no se limita a pegar debajo sino que alinea por nombres de columna

In [ ]:
import IPython.display as display
import pandas as pd

df1 = DataFrame({'clave': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data': range(7),
                 'otro': sample(range(10, 30), 7)})

df2 = DataFrame({'clave': ['e', 'e', 'e', 'e'],
                 'otro': sample(range(10, 30), 4),
                 'data': range(4) } )

df1.index = ["A"]*len(df1)
df2.index = ["B"]*len(df2)
display_side_by_side(df1,df2,pd.concat([df1,df2]), title="pd.concat([df1,df2])")

Con el parámetro axis (que por defecto vale 0) podemos hacer que en lugar de por filas concatene por columnas. En este caso lo lógico es que ambos dataframes tengan el mismo número de filas



In [ ]:
import numpy as np
filas = 5
df1 = DataFrame({'A': np.random.randint(1,10,filas) ,
                 'B': np.random.randint(1,10,filas),
                 'C': np.random.randint(1,10,filas)})

df2 = DataFrame({'D': np.random.randint(2000,3000,filas),
                 'E': np.random.randint(2000,3000,filas)})
display_side_by_side(df1,df2, pd.concat([df1,df2],axis=1),
                     title="pd.concat([df1,df2],axis=1)")

Un aspecto muy importante: igual que al concatenar utiliza los nombres de columna aquí va a usar los números de fila

In [ ]:
df2.index=[0,1,2,4,7]
display_side_by_side(df1,df2, pd.concat([df1,df2],axis=1),
                     title="pd.concat([df1,df2],axis=1)")

Si el número de filas o columnas no encaja, `concat` añadirá valores vacío para completar

In [ ]:
import numpy as np
filas = 5
df1 = DataFrame({'A': np.random.randint(1,10,filas) ,
                 'B': np.random.randint(1,10,filas),
                 'C': np.random.randint(1,10,filas)})

df2 = DataFrame({'D': np.random.randint(2000,3000,filas-1),
                 'E': np.random.randint(2000,3000,filas-1)})
display_side_by_side(df1,df2, pd.concat([df1,df2],axis=1),title="pd.concat([df1,df2],axis=1)")

display_side_by_side(df1,df2, pd.concat([df1,df2],axis=0),title="pd.concat([df1,df2],axis=0)")

**Ejercicio**  El siguiente código descarga tres ficheros con datos diarios de metereología en Madrid y sus metadatos a una carpeta ./raw

In [ ]:
import requests
from pathlib import Path
path = Path.cwd()
pathraw = Path(path,"raw")

pathraw.mkdir(exist_ok=True)

datos = ["https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/DatosMeteorologicos/Ficheros/Interpretaci%C3%B3n_datos_meteorologicos.pdf",
         "https://datos.madrid.es/egob/catalogo/300351-0-meteorologicos-diarios.csv",
         "https://datos.madrid.es/egob/catalogo/300351-3-meteorologicos-diarios.csv",
          "https://datos.madrid.es/egob/catalogo/300351-9-meteorologicos-diarios.csv"]
locales = []
for url in datos:
    nombre = Path(url).name
    camino = Path(pathraw,nombre)
    r = requests.get(url, allow_redirects=True) # el fichero queda en la variable r
    if r.status_code==200:
        with open(camino, 'wb') as f:
            f.write(r.content) # ahora lo grabamos localmente
        print("Grabado,",camino)
        locales.append(camino)
    else:
        print("Error descargando ",url)


In [ ]:
locales

Añadir código para combinar los 3 ficheros en uno solo sabiendo que tienen las mismas columnas y que sus caminos están en la variable `locales`

In [ ]:
# primero leemos todos los ficheros; dfs será una lista de dataframes
dfs = []
for p in locales:
    if p.suffix==".csv":
        df_temp = pd.read_csv(p,sep=";")
        dfs.append(df_temp)

In [ ]:
df = pd.concat(dfs)
df

In [ ]:
df.MAGNITUD.unique()


<a name="Merge"></a>
### Merge

En este caso se busca unir dos dataframes fijándonos en las coincidencias entre valores de dos columnas

In [ ]:
from pandas import DataFrame
from pandas import Series
import pandas as pd
from random import sample
df1 = DataFrame({'clave': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7),
                 'otro': sample(range(10, 30), 7)})

df2 = DataFrame({'clave': ['a', 'b', 'b', 'd'],
                 'data2': range(4)})
display_side_by_side(df1,df2)

Por defecto la mezcla es por la columna que se llama igual


In [ ]:

display_side_by_side(df1,df2, pd.merge(df1,df2),title="pd.merge(df1,df2)")


Si no se indica lo contrario, `merge` busca columnas comunes y hace un (inner) 'join'. Nótese que en este caso no se tienen en cuenta los índices
<br><br>

El método merge se puede llamar también dentro de un dataframe (es equivalente)

In [ ]:
df1.merge(df2)

También se pueden unir por varias columnas, que podemos especificar directamente con los parámetros `left_on`y `right_on`

In [ ]:

df3 = df1.merge(df2, left_on=['clave','data1'], right_on = ['clave','data2'])

display_side_by_side(df1,df2, df3)

Si la clave o claves por las que querenos unir se llaman ambas igual podemos usar simplemente `on`

In [ ]:
df3 = pd.merge(df1,df2,on='clave')
display_side_by_side(df1,df2, df3)

Además de *inner* join, se pueden hacer con el parámetro `how` tomando valores *left*, *right*, *outer*, *inner*

<img src="https://www.golinuxcloud.com/wp-content/uploads/types_joins-1320x961.png">

En el caso de left, right y full/outer si la columna no encaja se rellenan con valores NaN

In [ ]:
df3 = df1.merge( df2, on='clave', how='left')
display_side_by_side(df1,df2, df3)

**Ejercicio** En

https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/tweetsCompletadoOrdenRename.csv

Tenemos datos de tweets, incluyendo el identificador del usuario que que ha emitidos cada tweet,  `userid`.

En

https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/usersrentaf.csv

tenemos datos de usuarios: sú número de seguidores, la renta de la zona donde viven, etc. En este caso el identificador se llama simplemente `id`.

Queremos unir ambos ficheros, de forma que a cada tweet se le añadan los datos de su usuario. Si un tweet no tiene su usario en el segundo conjunto de datos debemos borrarlo. Igualmente si un usuario no tiene ningún tweet no se incluirá.





In [ ]:
import pandas as pd
url_tweets = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/tweetsCompletadoOrdenRename.csv"
url_users = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/usersrentaf.csv"
df_tweets = pd.read_csv(url_tweets)
df_users = pd.read_csv(url_users)



In [ ]:
df_tweets.columns